In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.neighbors import KNeighborsClassifier

In [52]:
data = pd.read_csv('data/movie_genre.csv')
data[0:10]

,movie,genre
0,''Buddhist Monkey'',Action
1,'...I Think You Went a Little Far with the Her...,Drama
2,'...nettirwnU',Drama
3,'04: How Was It for You?,Documentary
4,'05: How Was It for You?,Documentary
5,'06: The Big One,History
6,'15',Comedy
7,'1914' by 'Rita',Drama
8,'38,Drama
9,'44,Documentary


In [54]:
x = data['movie'].values
y = data['genre'].values
n = len(x)

x_train = x[:int(n/2)]
y_train = y[:int(n/2)]
x_val = x[int(n/2):]
y_val = y[int(n/2):]

26

26